<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Alpha_LSTM_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

incorporating some changes to LSTM.
Further improvements will be incorporated.
 Suggested by Deepthink
as instead of price changes, try to work on percent changes or log returns that are more stable to work with
Use GARCH (Generalized Autoregressive Conditional Heteroskedasticity) models to capture volatility patterns and incorporate them into your LSTM model.
Train on historical data and test on future data.

In [ ]:
!pip install ccxt pandas numpy scikit-learn tensorflow pandas-ta
!pip install arch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 9.6 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=639949210e76701fa068a00c8bdee13757a2ae11e74597c316e79ab314d83d7a
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 10.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize KuCoin exchange using CCXT
exchange = ccxt.kucoin()

# Fetch historical data from KuCoin
def fetch_historical_data(symbol, timeframe, limit=500):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logger.error(f"Error fetching data for {symbol}: {e}")
        return None

# Add technical indicators
def add_technical_indicators(df):
    df['RSI'] = ta.rsi(df['close'], length=14)  # RSI
    df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)  # ATR
    df['OBV'] = ta.obv(df['close'], df['volume'])  # OBV
    df['SMA_50'] = ta.sma(df['close'], length=50)  # Simple Moving Average (50)
    df['EMA_50'] = ta.ema(df['close'], length=50)  # Exponential Moving Average (50)
    df['MACD'] = ta.macd(df['close'])['MACD_12_26_9']  # MACD

    bbands = ta.bbands(df['close'], length=20)
    df['BB_upper'] = bbands['BBU_20_2.0']  # Upper Bollinger Band
    df['BB_middle'] = bbands['BBM_20_2.0']  # Middle Bollinger Band
    df['BB_lower'] = bbands['BBL_20_2.0']  # Lower Bollinger Band

    df = df.dropna()
    return df

# Preprocess data
def preprocess_data(df, sequence_length=50):
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50', 'MACD', 'BB_upper', 'BB_middle', 'BB_lower']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict price
def predict_price(symbol, sequence_length=50):
    df_1d = fetch_historical_data(symbol, '1d', limit=500)
    df_1w = fetch_historical_data(symbol, '1w', limit=500)

    if df_1d is None or df_1w is None:
        logger.error(f"Skipping {symbol} due to data fetch error.")
        return None

    df_1d = add_technical_indicators(df_1d)
    df_1w = add_technical_indicators(df_1w)

    X_1d, y_1d, scaler_1d = preprocess_data(df_1d, sequence_length)
    X_1w, y_1w, scaler_1w = preprocess_data(df_1w, sequence_length)

    model_1d = build_model((X_1d.shape[1], X_1d.shape[2]))
    model_1w = build_model((X_1w.shape[1], X_1w.shape[2]))

    model_1d.fit(X_1d, y_1d, epochs=10, batch_size=32, verbose=1, validation_split=0.2)
    model_1w.fit(X_1w, y_1w, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

    latest_sequence_1d = X_1d[-1].reshape(1, X_1d.shape[1], X_1d.shape[2])
    latest_sequence_1w = X_1w[-1].reshape(1, X_1w.shape[1], X_1w.shape[2])

    predicted_price_1d = scaler_1d.inverse_transform([[model_1d.predict(latest_sequence_1d)[0][0], 0, 0, 0, 0, 0, 0, 0, 0, 0]])[0][0]
    predicted_price_1w = scaler_1w.inverse_transform([[model_1w.predict(latest_sequence_1w)[0][0], 0, 0, 0, 0, 0, 0, 0, 0, 0]])[0][0]

    return {
        'symbol': symbol,
        'current_price': df_1d['close'].iloc[-1],
        '1d_prediction': predicted_price_1d,
        '1w_prediction': predicted_price_1w
    }

# Main function
symbols = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT' , 'NEAR/USDT' , 'FET/USDT' , 'LINK/USDT' , 'AVAX/USDT' , 'SOL/USDT' , 'INJ/USDT']  # Replace with desired trading pairs
results = []

for symbol in symbols:
    result = predict_price(symbol)
    if result:
        results.append(result)

# Display results
results_df = pd.DataFrame(results)
print(results_df)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.1013 - val_loss: 0.0679
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0098 - val_loss: 0.1421
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0086 - val_loss: 0.0705
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0059 - val_loss: 0.1208
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0054 - val_loss: 0.0730
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0062 - val_loss: 0.0878
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0049 - val_loss: 0.0769
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0050 - val_loss: 0.0949
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0045 - val_loss: 0.0578
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0051 - val_loss: 0.0749
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 117ms/step - loss: 0.0620 - val_loss: 0.0623
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0157 - val_loss: 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 0.1320 - val_loss: 0.0213
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0301 - val_loss: 0.0396
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0280 - val_loss: 0.0110
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0173 - val_loss: 0.0251
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0154 - val_loss: 0.0137
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0157 - val_loss: 0.0163
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0139 - val_loss: 0.0190
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0119 - val_loss: 0.0084
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0115 - val_loss: 0.0161
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0142 - val_loss: 0.0157
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - loss: 0.0833 - val_loss: 0.0139
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0250 - val_loss: 

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - loss: 0.1406 - val_loss: 0.0163
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0275 - val_loss: 0.0442
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0176 - val_loss: 0.0132
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0157 - val_loss: 0.0223
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0132 - val_loss: 0.0143
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0113 - val_loss: 0.0158
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0114 - val_loss: 0.0136
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0084 - val_loss: 0.0134
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0087 - val_loss: 0.0098
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0081 - val_loss: 0.0082
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 271ms/step - loss: 0.0300 - val_loss: 0.0090
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0166 - val_loss:

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.0795 - val_loss: 0.0686
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0166 - val_loss: 0.0537
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0111 - val_loss: 0.0470
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0096 - val_loss: 0.0620
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0073 - val_loss: 0.0388
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0072 - val_loss: 0.0455
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0049 - val_loss: 0.0411
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0060 - val_loss: 0.0416
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0056 - val_loss: 0.0397
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0048 - val_loss: 0.0269
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 290ms/step - loss: 0.3752 - val_loss: 0.4619
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0790 - val_loss: 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - loss: 0.0735 - val_loss: 0.0099
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0168 - val_loss: 0.0079
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0116 - val_loss: 0.0111
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0088 - val_loss: 0.0086
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0077 - val_loss: 0.0095
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0068 - val_loss: 0.0107
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0070 - val_loss: 0.0079
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0064 - val_loss: 0.0060
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0070 - val_loss: 0.0076
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0068 - val_loss: 0.0084
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - loss: 0.2098 - val_loss: 0.0341
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 0.0541 - val_loss:

percent prices model

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import logging
from arch import arch_model  # For GARCH modeling

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize KuCoin exchange using CCXT
exchange = ccxt.kucoin()

# Fetch historical data from KuCoin
def fetch_historical_data(symbol, timeframe, limit=500):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logger.error(f"Error fetching data for {symbol}: {e}")
        return None

# Add technical indicators
def add_technical_indicators(df):
    df['RSI'] = ta.rsi(df['close'], length=14)  # RSI
    df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)  # ATR
    df['OBV'] = ta.obv(df['close'], df['volume'])  # OBV
    df['SMA_50'] = ta.sma(df['close'], length=50)  # Simple Moving Average (50)
    df['EMA_50'] = ta.ema(df['close'], length=50)  # Exponential Moving Average (50)
    df = df.dropna()
    return df

# Add percent returns and GARCH volatility
def add_percent_returns_and_garch(df):
    df['percent_return'] = df['close'].pct_change() * 100
    df.dropna(inplace=True)

    # Fit GARCH model on percent returns
    try:
        garch_model = arch_model(df['percent_return'], vol='Garch', p=1, q=1)
        garch_fit = garch_model.fit(disp='off')
        df['garch_volatility'] = garch_fit.conditional_volatility
    except Exception as e:
        logger.error(f"Error fitting GARCH model: {e}")
        df['garch_volatility'] = np.nan

    df = df.dropna()
    return df

# Preprocess data
def preprocess_data(df, sequence_length=50):
    scaler = MinMaxScaler()
    features = df[['percent_return', 'garch_volatility', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']]
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the percent return
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict price using percent returns and GARCH
def predict_price_with_percent_returns(symbol, sequence_length=50):
    timeframes = ['15m', '4h', '1d', '1w']
    results = []

    for timeframe in timeframes:
        df = fetch_historical_data(symbol, timeframe, limit=500)

        if df is None:
            logger.error(f"Skipping {symbol} for {timeframe} due to data fetch error.")
            continue

        df = add_technical_indicators(df)
        df = add_percent_returns_and_garch(df)

        X, y, scaler = preprocess_data(df, sequence_length)
        model = build_model((X.shape[1], X.shape[2]))

        model.fit(X, y, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

        latest_sequence = X[-1].reshape(1, X.shape[1], X.shape[2])
        predicted_percent_return = model.predict(latest_sequence)[0][0]

        # Convert predicted percent returns back to prices
        last_close_price = df['close'].iloc[-1]
        predicted_price = last_close_price * (1 + predicted_percent_return / 100)

        results.append({
            'symbol': symbol,
            'timeframe': timeframe,
            'current_price': last_close_price,
            'predicted_price': predicted_price
        })

    return results

# Main function
symbols = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT']  # Replace with desired trading pairs
all_results = []

for symbol in symbols:
    results = predict_price_with_percent_returns(symbol)
    all_results.extend(results)

# Display results
results_df = pd.DataFrame(all_results)
print(results_df)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.07561. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - loss: 0.0768 - val_loss: 0.0127
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0161 - val_loss: 0.0132
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0156 - val_loss: 0.0125
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0127 - val_loss: 0.0124
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0124 - val_loss: 0.0123
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0108 - val_loss: 0.0120
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0102 - val_loss: 0.0120
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0099 - val_loss: 0.0118
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0114 - val_loss: 0.0117
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0113 - val_loss: 0.0117
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - loss: 0.0758 - val_loss: 0.0195
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0232 - val_loss: 0.0238
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0165 - val_loss: 0.0231
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0188 - val_loss: 0.0191
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0163 - val_loss: 0.0217
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0167 - val_loss: 0.0184
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0185 - val_loss: 0.0184
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0186 - val_loss: 0.0191
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0150 - val_loss: 0.0174
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0145 - val_loss: 0.0177
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
Epoch 1/10


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - loss: 0.2041 - val_loss: 0.0202
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0201 - val_loss: 0.0150
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0204 - val_loss: 0.0081
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0198 - val_loss: 0.0067
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0156 - val_loss: 0.0069
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0137 - val_loss: 0.0074
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0159 - val_loss: 0.0067
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0163 - val_loss: 0.0067
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0125 - val_loss: 0.0066
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0123 - val_loss: 0.0065
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - loss: 0.1109 - val_loss: 0.1073
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0194 - val_loss: 0.0245
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0183 - val_loss: 0.0287
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0129 - val_loss: 0.0242
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0131 - val_loss: 0.0233
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0116 - val_loss: 0.0236
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0112 - val_loss: 0.0233
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0110 - val_loss: 0.0229
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0123 - val_loss: 0.0229
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0122 - val_loss: 0.0235
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 262ms/step - loss: 0.0350 - val_loss: 0.0114
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0166 - val_loss: 0.0096
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0141 - val_loss: 0.0114
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0092 - val_loss: 0.0135
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0099 - val_loss: 0.0099
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0080 - val_loss: 0.0089
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0093 - val_loss: 0.0090
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0066 - val_loss: 0.0088
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0082 - val_loss: 0.0087
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0078 - val_loss: 0.0089
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
      symbol timeframe  current_price  predicted_price
0   BTC/USDT       15m    105165.7000    105618.052929
1   BTC/USDT        4h    1051